In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat May 20 16:02:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
%%capture
!pip install keras_applications
!pip install keras_vggface
filename = "/usr/local/lib/python3.10/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))
!pip install mtcnn

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error

from keras.utils import load_img, img_to_array

from sklearn.metrics import accuracy_score

from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/data.csv')
data.head()

,Unnamed: 0,bmi,gender,is_training,name
0,0,34.207396,Male,1,img_0.bmp
1,1,26.453720,Male,1,img_1.bmp
2,2,34.967561,Female,1,img_2.bmp
3,3,22.044766,Female,1,img_3.bmp
4,4,37.758789,Female,1,img_4.bmp


## MTCNN to crop the images and scale them equally

In [ ]:
import os
import cv2
from mtcnn import MTCNN
from keras.layers.pooling import GlobalAveragePooling2D
import pandas as pd
from keras.utils.layer_utils import get_source_inputs
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Dense, Flatten, Conv2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


# Load the CSV data
data = pd.read_csv('/content/drive/MyDrive/data.csv')

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Set up the training data generator
directory = '/content/drive/MyDrive/BMI/Images'

# Create a new folder to save the cropped face images
cropped_directory = '/content/drive/MyDrive/BMI_Cropped/CroppedImages'
os.makedirs(cropped_directory, exist_ok=True)

# Create an instance of MTCNN
mtcnn = MTCNN()

# Function to crop faces from images and save them to a new folder
def save_cropped_faces(image_path, save_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Invalid image:", image_path)
            return
        
        result = mtcnn.detect_faces(image)
        if result:
            x, y, w, h = result[0]['box']
        
            # Calculate the aspect ratio
            aspect_ratio = float(w) / h
            
            # Determine the new width and height while maintaining the aspect ratio
            if aspect_ratio > 1.0:
                new_width = int(224 * aspect_ratio)
                x = x - (new_width - w) // 2
                w = new_width
            else:
                new_height = int(224 / aspect_ratio)
                y = y - (new_height - h) // 2
                h = new_height
            
            # Crop the face region while maintaining the aspect ratio
            face = image[max(0, y):y+h, max(0, x):x+w]

            # Resize the face region to fit the desired padding size
            # resized_face = cv2.resize(face, (224, 224))

            # Create a padded canvas
            try:
                padded_image = np.zeros((224, 224, 3), dtype=np.uint8)
                padded_image[:face.shape[0], :face.shape[1]] = face
            except:
                height = face.shape[0]  # Get the height of the image
                width = face.shape[1]   # Get the width of the image
                scale_factor = min(224 / width, 224 / height)

                # Resize the image while preserving aspect ratio
                new_width = int(width * scale_factor)
                new_height = int(height * scale_factor)
                face = cv2.resize(face, (new_width, new_height))
                padded_image = np.zeros((224, 224, 3), dtype=np.uint8)
                padded_image[:face.shape[0], :face.shape[1]] = face
            cv2.imwrite(save_path, padded_image)
    except Exception as e:
        print("Error processing image:", image_path)
        print(str(e))


# Iterate through the data and crop faces from images
for index, row in data.iterrows():
    image_path = os.path.join(directory, row['name'])
    save_path = os.path.join(cropped_directory, row['name'])
    save_cropped_faces(image_path, save_path)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 20ms/step
Invalid image: /content/drive/MyDrive/BMI/Images/img_3660.bmp
1/1 [==============================] - 0s 23ms/step
Invalid image: /content/drive/MyDrive/BMI/Images/img_3680.bmp
Invalid image: /content/drive/MyDrive/BMI/Images/img_3681.bmp
1/1 [==============================] - 0s 26ms/step
Invalid image: /content/drive/MyDrive/BMI/Images/img_3694.bmp
Invalid image: /content/drive/MyDrive/BMI/Images/img_3695.bmp
1/1 [==============================] - 0s 21ms/step
Invalid image: /content/drive/MyDrive/BMI/Images/img_3726.bmp
Invalid image: /content/drive/MyDrive/BMI/Images/img_3727.bmp
1/1 [==============================] - 0s 23ms/step
Invalid image: /content/drive/MyDrive/BMI/Images/img_3731.bmp
1/1 [==============================] - 0s 21ms/step
Invalid image: /content/drive/MyDrive/BMI/Images/img_3764.bmp
Invalid image: /content/drive/MyDrive/BMI/Images/img_3765.bmp
1/1 [============

## Trying out diferent models

In [ ]:
# Set up the training data generator with the cropped images
train_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='training')

validation_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='validation')

print("Image shape:", train_set.image_shape)

# Load the VGGFace model without the top layer
base_model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.layers[-1].output
x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
x = keras.layers.AveragePooling2D()(x)
x = Dense(units=128, activation='relu')(x)
predictions = Dense(1, activation='relu')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

# Set up callbacks to save the best model and enable early stopping
checkpoint_bmi = ModelCheckpoint('best_bmi_model_v2.h5', monitor='val_rmse', save_best_only=True, verbose=1)

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    train_set,
    epochs=50,
    callbacks=[checkpoint_bmi], validation_data=validation_set
)

train_predictions = model.predict(train_set)

Found 2805 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Found 701 validated image filenames.
Image shape: (224, 224, 3)
Model: "model_88"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_89 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                            

In [ ]:
def cropped_faces(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Invalid image:", image_path)
            return None
        
        result = mtcnn.detect_faces(image)
        if result:
            x, y, w, h = result[0]['box']
        
            # Calculate the aspect ratio
            aspect_ratio = float(w) / h
            
            # Determine the new width and height while maintaining the aspect ratio
            if aspect_ratio > 1.0:
                new_width = int(224 * aspect_ratio)
                x = x - (new_width - w) // 2
                w = new_width
            else:
                new_height = int(224 / aspect_ratio)
                y = y - (new_height - h) // 2
                h = new_height
            
            # Crop the face region while maintaining the aspect ratio
            face = image[max(0, y):y+h, max(0, x):x+w]

            # Resize the face region to fit the desired padding size
            resized_face = cv2.resize(face, (224, 224))

            # Create a padded canvas
            padded_image = np.zeros((224, 224, 3), dtype=np.uint8)
            padded_image[:resized_face.shape[0], :resized_face.shape[1]] = resized_face

            # Save the padded image
            cropped_image_path = '/content/drive/MyDrive/Cropped_Image.jpg'
            cv2.imwrite(cropped_image_path, padded_image)
            
            return cropped_image_path
        else:
            print("No face detected in the image:", image_path)
            return None

    except Exception as e:
        print("Error processing image:", image_path)
        print(str(e))
        return None

# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/PassportPhoto.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)

# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/PP_1.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)


# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/Test1.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)


# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/Test2.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)







# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/Test1.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/Test2.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/PP_1.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

1/1 [==============================] - 0s 21ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 23ms/step
Predicted BMI pi: [[32.671383]
 [32.61521 ]
 [32.95267 ]]
1/1 [==============================] - 0s 22ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 26ms/step
Predicted BMI pi: [[31.611036]
 [31.86374 ]
 [33.43963 ]]
1/1 [==============================] - 0s 20ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 22ms/step
Predicted BMI pi: [[33.532097]
 [31.198223]
 [31.872921]]
1/1 [==============================] - 0s 20ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 22ms/step
Predicted BMI pi: [[32.53389 ]
 [31.567865]
 [32.25808 ]]
1/1 [==============================] - 0s 24ms/step
Predicted BMI: [[32.521862]
 [32.72001 ]
 [32.67032 ]]
1/1 [==============================] - 0s 24ms/step
Predicted BMI: [[32.3

In [ ]:
# Set up the training data generator with the cropped images
train_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='training')

validation_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='validation')

print("Image shape:", train_set.image_shape)

# Load the VGGFace model without the top layer
base_model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.layers[-1].output
x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
x = keras.layers.AveragePooling2D()(x)
x = Dense(units=128, activation='relu')(x)
predictions = Dense(1, activation='relu')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

# Set up callbacks to save the best model and enable early stopping
checkpoint_bmi = ModelCheckpoint('best_bmi_model_v2_1.h5', monitor='val_rmse', save_best_only=True, verbose=1)

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    train_set,
    epochs=50,
    callbacks=[checkpoint_bmi], validation_data=validation_set
)

train_predictions = model.predict(train_set)

Found 2805 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Found 701 validated image filenames.
Image shape: (224, 224, 3)
Model: "model_89"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_90 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                            

In [ ]:
def cropped_faces(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Invalid image:", image_path)
            return None
        
        result = mtcnn.detect_faces(image)
        if result:
            x, y, w, h = result[0]['box']
        
            # Calculate the aspect ratio
            aspect_ratio = float(w) / h
            
            # Determine the new width and height while maintaining the aspect ratio
            if aspect_ratio > 1.0:
                new_width = int(224 * aspect_ratio)
                x = x - (new_width - w) // 2
                w = new_width
            else:
                new_height = int(224 / aspect_ratio)
                y = y - (new_height - h) // 2
                h = new_height
            
            # Crop the face region while maintaining the aspect ratio
            face = image[max(0, y):y+h, max(0, x):x+w]

            # Resize the face region to fit the desired padding size
            resized_face = cv2.resize(face, (224, 224))

            # Create a padded canvas
            padded_image = np.zeros((224, 224, 3), dtype=np.uint8)
            padded_image[:resized_face.shape[0], :resized_face.shape[1]] = resized_face

            # Save the padded image
            cropped_image_path = '/content/drive/MyDrive/Cropped_Image.jpg'
            cv2.imwrite(cropped_image_path, padded_image)
            
            return cropped_image_path
        else:
            print("No face detected in the image:", image_path)
            return None

    except Exception as e:
        print("Error processing image:", image_path)
        print(str(e))
        return None

# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/PassportPhoto.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)

# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/PP_1.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)


# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/Test1.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)


# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/Test2.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)







# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/Test1.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/Test2.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/PP_1.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

1/1 [==============================] - 0s 21ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 188ms/step
Predicted BMI pi: [[32.553722]
 [32.78946 ]
 [32.862858]]
1/1 [==============================] - 0s 21ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 24ms/step
Predicted BMI pi: [[32.68963 ]
 [32.761345]
 [32.71654 ]]
1/1 [==============================] - 0s 21ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 23ms/step
Predicted BMI pi: [[32.53496]
 [32.45109]
 [32.72851]]
1/1 [==============================] - 0s 21ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 22ms/step
Predicted BMI pi: [[32.522846]
 [32.50012 ]
 [32.68018 ]]
1/1 [==============================] - 0s 23ms/step
Predicted BMI: [[32.495083]
 [32.49697 ]
 [32.81796 ]]
1/1 [==============================] - 0s 24ms/step
Predicted BMI: [[32.647

In [ ]:
# Set up the training data generator with the cropped images
train_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='training')

validation_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='validation')

print("Image shape:", train_set.image_shape)

# Load the VGGFace model without the top layer
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.layers[-1].output
# x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
# x = keras.layers.AveragePooling2D()(x)
x = Dense(units=128, activation='relu')(x)
predictions = Dense(1, activation='relu')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

# Set up callbacks to save the best model and enable early stopping
checkpoint_bmi = ModelCheckpoint('best_bmi_model_v2_2.h5', monitor='val_rmse', save_best_only=True, verbose=1)

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    train_set,
    epochs=50,
    callbacks=[checkpoint_bmi], validation_data=validation_set
)

train_predictions = model.predict(train_set)

Found 2805 validated image filenames.
Found 701 validated image filenames.
Image shape: (224, 224, 3)
Model: "model_90"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_94 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_94[0][0]']               
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]'] 

In [ ]:
def cropped_faces(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Invalid image:", image_path)
            return None
        
        result = mtcnn.detect_faces(image)
        if result:
            x, y, w, h = result[0]['box']
        
            # Calculate the aspect ratio
            aspect_ratio = float(w) / h
            
            # Determine the new width and height while maintaining the aspect ratio
            if aspect_ratio > 1.0:
                new_width = int(224 * aspect_ratio)
                x = x - (new_width - w) // 2
                w = new_width
            else:
                new_height = int(224 / aspect_ratio)
                y = y - (new_height - h) // 2
                h = new_height
            
            # Crop the face region while maintaining the aspect ratio
            face = image[max(0, y):y+h, max(0, x):x+w]

            # Resize the face region to fit the desired padding size
            resized_face = cv2.resize(face, (224, 224))

            # Create a padded canvas
            padded_image = np.zeros((224, 224, 3), dtype=np.uint8)
            padded_image[:resized_face.shape[0], :resized_face.shape[1]] = resized_face

            # Save the padded image
            cropped_image_path = '/content/drive/MyDrive/Cropped_Image.jpg'
            cv2.imwrite(cropped_image_path, padded_image)
            
            return cropped_image_path
        else:
            print("No face detected in the image:", image_path)
            return None

    except Exception as e:
        print("Error processing image:", image_path)
        print(str(e))
        return None

# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/PassportPhoto.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)

# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/PP_1.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)


# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/Test1.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)


# Load the new image and perform face cropping
image_path = '/content/drive/MyDrive/Test_Image/Test2.jpg' 
cropped_image_path = cropped_faces(image_path)

if cropped_image_path is not None:
    # Load the cropped image and preprocess
    img = load_img(cropped_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale the pixel values
    print("Preprocessed image shape:", img_array.shape)
else:
    print("Failed to perform face cropping.")


# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI pi:", predicted_bmi)







# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/Test1.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/Test2.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

# Load the new image
image_path = '/content/drive/MyDrive/Test_Image/PP_1.jpg'  
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale the pixel values

# Predict the BMI for the new image
prediction = model.predict(img_array)
predicted_bmi = prediction[0][0]
print("Predicted BMI:", predicted_bmi)

1/1 [==============================] - 0s 21ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 1s 1s/step
Predicted BMI pi: [[34.134846]]
1/1 [==============================] - 0s 21ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 25ms/step
Predicted BMI pi: [[33.83492]]
1/1 [==============================] - 0s 22ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 25ms/step
Predicted BMI pi: [[33.8629]]
1/1 [==============================] - 0s 23ms/step
Preprocessed image shape: (1, 224, 224, 3)
1/1 [==============================] - 0s 25ms/step
Predicted BMI pi: [[33.84612]]
1/1 [==============================] - 0s 26ms/step
Predicted BMI: [[33.396328]]
1/1 [==============================] - 0s 24ms/step
Predicted BMI: [[33.75414]]
1/1 [==============================] - 0s 25ms/step
Predicted BMI: [[33.79772]]


In [ ]:
# Set up the training data generator with the cropped images
train_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='training')

validation_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='validation')

print("Image shape:", train_set.image_shape)

# Load the VGGFace model without the top layer
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.layers[-1].output
# x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
# x = keras.layers.AveragePooling2D()(x)
x = Dense(units=128, activation='relu')(x)
predictions = Dense(1, activation='relu')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

# Set up callbacks to save the best model and enable early stopping
checkpoint_bmi = ModelCheckpoint('best_bmi_model_v2_2.h5', monitor='val_rmse', save_best_only=True, verbose=1)

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    train_set,
    epochs=50,
    callbacks=[checkpoint_bmi], validation_data=validation_set
)

train_predictions = model.predict(train_set)

Found 2805 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Found 701 validated image filenames.
Image shape: (224, 224, 3)
Model: "model_91"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_95 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_95[0][0]']               
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                    

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from keras_vggface.vggface import VGGFace

# Set up the training data generator with the cropped images
train_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='training')

validation_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='bmi',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    subset='validation')

print("Image shape:", train_set.image_shape)

# Load the VGGFace model without the top layer
base_model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.layers[-1].output
x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
x = keras.layers.AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)
predictions = Dense(1, activation='linear')(x)  # Use linear activation for BMI prediction

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Define the custom loss function
def custom_loss(y_true, y_pred):
    penalty = 10.0  # Penalty factor for overestimation
    squared_difference = tf.square(y_true - y_pred)
    overestimation_penalty = tf.maximum(0.0, y_pred - y_true) * penalty
    return tf.reduce_mean(squared_difference + overestimation_penalty, axis=-1)

# Compile the model with the custom loss function
model.compile(optimizer='adam', loss=custom_loss, metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

# Set up callbacks to save the best model and enable early stopping
checkpoint_bmi = ModelCheckpoint('best_bmi_model_v3_0.h5', monitor='val_rmse', save_best_only=True, verbose=1)

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    train_set,
    epochs=50,
    callbacks=[checkpoint_bmi],
    validation_data=validation_set
)

train_predictions = model.predict(train_set)


Found 2805 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Found 701 validated image filenames.
Image shape: (224, 224, 3)
Model: "model_106"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_110 (InputLayer)      [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                           

## Final comparison of VGGFace models for predicting BMI below

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras_vggface.vggface import VGGFace

def cropped_faces(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Invalid image:", image_path)
            return None
        
        result = mtcnn.detect_faces(image)
        if result:
            x, y, w, h = result[0]['box']
        
            # Calculate the aspect ratio
            aspect_ratio = float(w) / h
            
            # Determine the new width and height while maintaining the aspect ratio
            if aspect_ratio > 1.0:
                new_width = int(224 * aspect_ratio)
                x = x - (new_width - w) // 2
                w = new_width
            else:
                new_height = int(224 / aspect_ratio)
                y = y - (new_height - h) // 2
                h = new_height
            
            # Crop the face region while maintaining the aspect ratio
            face = image[max(0, y):y+h, max(0, x):x+w]

            # Resize the face region to fit the desired padding size
            resized_face = cv2.resize(face, (224, 224))

            # Create a padded canvas
            padded_image = np.zeros((224, 224, 3), dtype=np.uint8)
            padded_image[:resized_face.shape[0], :resized_face.shape[1]] = resized_face

            # Save the padded image
            cropped_image_path = '/content/drive/MyDrive/Cropped_Image.jpg'
            cv2.imwrite(cropped_image_path, padded_image)
            
            return cropped_image_path
        else:
            print("No face detected in the image:", image_path)
            return None

    except Exception as e:
        print("Error processing image:", image_path)
        print(str(e))
        return None

i = 0
for i in range(10):
    

    # Set up the training data generator with the cropped images
    train_set = train_datagen.flow_from_dataframe(
        dataframe=data,
        directory=cropped_directory,
        x_col='name',
        y_col='bmi',
        target_size=(224, 224),
        batch_size=32,
        class_mode='other',
        subset='training')

    validation_set = train_datagen.flow_from_dataframe(
        dataframe=data,
        directory=cropped_directory,
        x_col='name',
        y_col='bmi',
        target_size=(224, 224),
        batch_size=32,
        class_mode='other',
        subset='validation')

    print("Image shape:", train_set.image_shape)

    # Load the VGGFace model without the top layer
    base_model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3))

    # Add custom layers on top of the base model
    x = base_model.layers[-1].output
    x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
    x = keras.layers.AveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(units=128, activation='relu')(x)
    predictions = Dense(1, activation='linear')(x)  # Use linear activation for BMI prediction

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    if i == 0:
        for layer in base_model.layers:
            layer.trainable = False
    else:
        for layer in base_model.layers[:-i]:
            layer.trainable = False

    # # Define the custom loss function
    # def custom_loss(y_true, y_pred):
    #     penalty = 10.0  # Penalty factor for overestimation
    #     squared_difference = tf.square(y_true - y_pred)
    #     overestimation_penalty = tf.maximum(0.0, y_pred - y_true) * penalty
    #     return tf.reduce_mean(squared_difference + overestimation_penalty, axis=-1)

    def custom_loss(y_true, y_pred):
        penalty_overestimation_10 = 10.0  # Penalty factor for overestimation
        penalty_underestimation_10 = 10.0  # Penalty factor for underestimation by 10
        penalty_overestimation_5 = 5.0  # Penalty factor for overestimation
        penalty_underestimation_5 = 5.0  # Penalty factor for underestimation by 5
        
        squared_difference = tf.square(y_true - y_pred)
        overestimation_penalty_10 = tf.maximum(0.0, y_pred - y_true) * penalty_overestimation_10
        underestimation_penalty_10 = tf.maximum(0.0, y_true - y_pred - 10) * penalty_underestimation_10
        underestimation_penalty_5 = tf.maximum(0.0, y_true - y_pred - 5) * penalty_underestimation_5
        overestimation_penalty_5 = tf.maximum(0.0, y_pred - y_true) * penalty_overestimation_5
        total_loss = tf.reduce_mean(squared_difference + overestimation_penalty_10 + overestimation_penalty_5 + underestimation_penalty_10 + underestimation_penalty_5, axis=-1)
        
        return total_loss

    # Register the custom loss function
    keras.utils.get_custom_objects()['custom_loss'] = custom_loss

    # Compile the model with the custom loss function
    model.compile(optimizer='adam', loss=custom_loss, metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

    # Set up callbacks to save the best model and enable early stopping
    checkpoint_bmi = ModelCheckpoint('best_bmi_model_v3_'+str(i)+'.h5', monitor='val_rmse', save_best_only=True, verbose=1)

    # Print the model summary
    model.summary()

    # Train the model
    history = model.fit(
        train_set,
        epochs=50,
        callbacks=[checkpoint_bmi],
        validation_data=validation_set
    )
    print("Checking with unfreezing last " + str(i) + " layers: ")
    train_predictions = model.predict(train_set)

    # Load the new image and perform face cropping
    image_path = '/content/drive/MyDrive/Test_Image/PassportPhoto.jpg' 
    cropped_image_path = cropped_faces(image_path)

    if cropped_image_path is not None:
        # Load the cropped image and preprocess
        img = load_img(cropped_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale the pixel values
        print("Preprocessed image shape:", img_array.shape)
    else:
        print("Failed to perform face cropping.")


    # Predict the BMI for the new image
    prediction = model.predict(img_array)
    predicted_bmi = prediction[0][0]
    print("Predicted BMI pi:", predicted_bmi)

    # Load the new image and perform face cropping
    image_path = '/content/drive/MyDrive/Test_Image/PP_1.jpg' 
    cropped_image_path = cropped_faces(image_path)

    if cropped_image_path is not None:
        # Load the cropped image and preprocess
        img = load_img(cropped_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale the pixel values
        print("Preprocessed image shape:", img_array.shape)
    else:
        print("Failed to perform face cropping.")


    # Predict the BMI for the new image
    prediction = model.predict(img_array)
    predicted_bmi = prediction[0][0]
    print("Predicted BMI pi:", predicted_bmi)


    # Load the new image and perform face cropping
    image_path = '/content/drive/MyDrive/Test_Image/Test1.jpg' 
    cropped_image_path = cropped_faces(image_path)

    if cropped_image_path is not None:
        # Load the cropped image and preprocess
        img = load_img(cropped_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale the pixel values
        print("Preprocessed image shape:", img_array.shape)
    else:
        print("Failed to perform face cropping.")


    # Predict the BMI for the new image
    prediction = model.predict(img_array)
    predicted_bmi = prediction[0][0]
    print("Predicted BMI pi:", predicted_bmi)


    # Load the new image and perform face cropping
    image_path = '/content/drive/MyDrive/Test_Image/Test2.jpg' 
    cropped_image_path = cropped_faces(image_path)

    if cropped_image_path is not None:
        # Load the cropped image and preprocess
        img = load_img(cropped_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale the pixel values
        print("Preprocessed image shape:", img_array.shape)
    else:
        print("Failed to perform face cropping.")


    # Predict the BMI for the new image
    prediction = model.predict(img_array)
    predicted_bmi = prediction[0][0]
    print("Predicted BMI pi:", predicted_bmi)

    print("\n\n")
    


Found 2805 validated image filenames.
Found 701 validated image filenames.
Image shape: (224, 224, 3)
Model: "model_112"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_116 (InputLayer)      [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                     

### Predict Gender

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras_vggface.vggface import VGGFace

def cropped_faces(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Invalid image:", image_path)
            return None
        
        result = mtcnn.detect_faces(image)
        if result:
            x, y, w, h = result[0]['box']
        
            # Calculate the aspect ratio
            aspect_ratio = float(w) / h
            
            # Determine the new width and height while maintaining the aspect ratio
            if aspect_ratio > 1.0:
                new_width = int(224 * aspect_ratio)
                x = x - (new_width - w) // 2
                w = new_width
            else:
                new_height = int(224 / aspect_ratio)
                y = y - (new_height - h) // 2
                h = new_height
            
            # Crop the face region while maintaining the aspect ratio
            face = image[max(0, y):y+h, max(0, x):x+w]

            # Resize the face region to fit the desired padding size
            resized_face = cv2.resize(face, (224, 224))

            # Create a padded canvas
            padded_image = np.zeros((224, 224, 3), dtype=np.uint8)
            padded_image[:resized_face.shape[0], :resized_face.shape[1]] = resized_face

            # Save the padded image
            cropped_image_path = '/content/drive/MyDrive/Cropped_Image.jpg'
            cv2.imwrite(cropped_image_path, padded_image)
            
            return cropped_image_path
        else:
            print("No face detected in the image:", image_path)
            return None

    except Exception as e:
        print("Error processing image:", image_path)
        print(str(e))
        return None
cropped_directory = '/content/drive/MyDrive/BMI_Cropped/CroppedImages'
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_set = train_datagen.flow_from_dataframe(
        dataframe=data,
        directory=cropped_directory,
        x_col='name',
        y_col='gender',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training')

# Set up the training data generator with the cropped images
train_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='gender',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='gender',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

print("Image shape:", train_set.image_shape)
i = 0
for i in range(5):
    # Load the VGGFace model without the top layer
    base_model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3))

    # Add custom layers on top of the base model
    x = base_model.layers[-1].output
    x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
    x = keras.layers.AveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(units=128, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)  # Use linear activation for BMI prediction

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    if i == 0:
        for layer in base_model.layers:
            layer.trainable = False
    else:
        for layer in base_model.layers[:-i]:
            layer.trainable = False

    # # Define the custom loss function
    # def custom_loss(y_true, y_pred):
    #     penalty = 10.0  # Penalty factor for overestimation
    #     squared_difference = tf.square(y_true - y_pred)
    #     overestimation_penalty = tf.maximum(0.0, y_pred - y_true) * penalty
    #     return tf.reduce_mean(squared_difference + overestimation_penalty, axis=-1)

    # Register the custom loss function
    # keras.utils.get_custom_objects()['custom_loss'] = custom_loss

    # Compile the model with the custom loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Set up callbacks to save the best model and enable early stopping
    checkpoint_gender = ModelCheckpoint('best_gender_model_v1_'+str(i)+'.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

    # Print the model summary
    model.summary()

    # Train the model
    history = model.fit(
        train_set,
        epochs=50,
        callbacks=[checkpoint_gender],
        validation_data=validation_set
    )
    print("Checking with unfreezing last " + str(i) + " layers: ")
    train_predictions = model.predict(train_set)

    # Load the new image and perform face cropping
    image_path = '/content/drive/MyDrive/Test_Image/PassportPhoto.jpg' 
    cropped_image_path = cropped_faces(image_path)

    if cropped_image_path is not None:
        # Load the cropped image and preprocess
        img = load_img(cropped_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale the pixel values
        print("Preprocessed image shape:", img_array.shape)
    else:
        print("Failed to perform face cropping.")


    # Predict the BMI for the new image
    prediction = model.predict(img_array)
    predicted_bmi = prediction[0][0]
    print("Predicted Gender pi:", predicted_bmi)

    # Load the new image and perform face cropping
    image_path = '/content/drive/MyDrive/Test_Image/PP_1.jpg' 
    cropped_image_path = cropped_faces(image_path)

    if cropped_image_path is not None:
        # Load the cropped image and preprocess
        img = load_img(cropped_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale the pixel values
        print("Preprocessed image shape:", img_array.shape)
    else:
        print("Failed to perform face cropping.")


    # Predict the BMI for the new image
    prediction = model.predict(img_array)
    predicted_bmi = prediction[0][0]
    print("Predicted Gender pi:", predicted_bmi)


    # Load the new image and perform face cropping
    image_path = '/content/drive/MyDrive/Test_Image/Test1.jpg' 
    cropped_image_path = cropped_faces(image_path)

    if cropped_image_path is not None:
        # Load the cropped image and preprocess
        img = load_img(cropped_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale the pixel values
        print("Preprocessed image shape:", img_array.shape)
    else:
        print("Failed to perform face cropping.")


    # Predict the BMI for the new image
    prediction = model.predict(img_array)
    predicted_bmi = prediction[0][0]
    print("Predicted Gender pi:", predicted_bmi)


    # Load the new image and perform face cropping
    image_path = '/content/drive/MyDrive/Test_Image/Test2.jpg' 
    cropped_image_path = cropped_faces(image_path)

    if cropped_image_path is not None:
        # Load the cropped image and preprocess
        img = load_img(cropped_image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale the pixel values
        print("Preprocessed image shape:", img_array.shape)
    else:
        print("Failed to perform face cropping.")


    # Predict the BMI for the new image
    prediction = model.predict(img_array)
    predicted_bmi = prediction[0][0]
    print("Predicted Gender pi:", predicted_bmi)

    print("\n\n")
    


Found 2805 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Found 2805 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Found 701 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Image shape: (224, 224, 3)
58909280/58909280 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                   

KeyboardInterrupt: ignored

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras_vggface.vggface import VGGFace

def cropped_faces(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Invalid image:", image_path)
            return None
        
        result = mtcnn.detect_faces(image)
        if result:
            x, y, w, h = result[0]['box']
        
            # Calculate the aspect ratio
            aspect_ratio = float(w) / h
            
            # Determine the new width and height while maintaining the aspect ratio
            if aspect_ratio > 1.0:
                new_width = int(224 * aspect_ratio)
                x = x - (new_width - w) // 2
                w = new_width
            else:
                new_height = int(224 / aspect_ratio)
                y = y - (new_height - h) // 2
                h = new_height
            
            # Crop the face region while maintaining the aspect ratio
            face = image[max(0, y):y+h, max(0, x):x+w]

            # Resize the face region to fit the desired padding size
            resized_face = cv2.resize(face, (224, 224))

            # Create a padded canvas
            padded_image = np.zeros((224, 224, 3), dtype=np.uint8)
            padded_image[:resized_face.shape[0], :resized_face.shape[1]] = resized_face

            # Save the padded image
            cropped_image_path = '/content/drive/MyDrive/Cropped_Image.jpg'
            cv2.imwrite(cropped_image_path, padded_image)
            
            return cropped_image_path
        else:
            print("No face detected in the image:", image_path)
            return None

    except Exception as e:
        print("Error processing image:", image_path)
        print(str(e))
        return None
cropped_directory = '/content/drive/MyDrive/BMI_Cropped/CroppedImages'
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_set = train_datagen.flow_from_dataframe(
        dataframe=data,
        directory=cropped_directory,
        x_col='name',
        y_col='gender',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training')

# Set up the training data generator with the cropped images
train_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='gender',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_set = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=cropped_directory,
    x_col='name',
    y_col='gender',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

print("Image shape:", train_set.image_shape)
# i = 0
# for i in range(5):
# Load the VGGFace model without the top layer
base_model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.layers[-1].output
x = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(x)
x = keras.layers.AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # Use linear activation for BMI prediction

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
# if i == 0:
#     for layer in base_model.layers:
#         layer.trainable = False
# else:
for layer in base_model.layers[:-4]:
    layer.trainable = False

# # Define the custom loss function
# def custom_loss(y_true, y_pred):
#     penalty = 10.0  # Penalty factor for overestimation
#     squared_difference = tf.square(y_true - y_pred)
#     overestimation_penalty = tf.maximum(0.0, y_pred - y_true) * penalty
#     return tf.reduce_mean(squared_difference + overestimation_penalty, axis=-1)

# Register the custom loss function
# keras.utils.get_custom_objects()['custom_loss'] = custom_loss

# Compile the model with the custom loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks to save the best model and enable early stopping
checkpoint_gender = ModelCheckpoint('best_gender_model_v1_4.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    train_set,
    epochs=50,
    callbacks=[checkpoint_gender],
    validation_data=validation_set
)

    


Found 2805 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Found 2805 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Found 701 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 700 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


Image shape: (224, 224, 3)
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (No

NameError: ignored